In [1]:
styleimg = 'base.jpg'
myimg = 'ashwin.jpg'

In [2]:
ouputimg = 'cubeash.jpg'

In [3]:
from keras import backend as K
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img, img_to_array

Using TensorFlow backend.


In [4]:
targetHeight = 512 
targetWidth = 512
targetSize = (targetHeight, targetWidth)

In [7]:
import numpy as np

In [8]:
cImage = load_img(path=myimg, target_size=targetSize)
cImArr = img_to_array(cImage)
cImArr = K.variable(preprocess_input(np.expand_dims(cImArr, axis=0)), dtype='float32')

In [9]:
sImage = load_img(path=styleimg, target_size=targetSize)
sImArr = img_to_array(sImage)
sImArr = K.variable(preprocess_input(np.expand_dims(sImArr, axis=0)), dtype='float32')

In [11]:
gIm0 = np.random.randint(256, size=(targetWidth, targetHeight, 3)).astype('float64')
gIm0 = preprocess_input(np.expand_dims(gIm0, axis=0))
gImPlaceholder = K.placeholder(shape=(1, targetWidth, targetHeight, 3))

In [12]:
def get_feature_reps(x, layer_names, model):
    featMatrices = []
    for ln in layer_names:
        selectedLayer = model.get_layer(ln)
        featRaw = selectedLayer.output
        featRawShape = K.shape(featRaw).eval(session=tf_session)
        
        N_1 = featRawShape[-1]
        M_1 = featRawShape[1] * featRawShape[2]
        
        featMatrix = K.reshape(featRaw, (M_1, N_1))
        featMatrix = K.transpose(featMatrix)
        featMatrices.append(featMatrix)
    return featMatrices
        


In [14]:
def get_content_loss(F, P):
    cLoss = 0.5*K.sum(K.sqaure(F - P))
    return cLoss

In [15]:
def get_Gram_matrix(F):
    G = K.dot(F, K.transpose(F))
    return G

In [16]:
def get_style_loss(ws, Gs, As):
    sLoss = K.variable(0.)
    for w,G,A in zip(ws, Gs, As):
        M_1 = K.int_shape(G)[1]
        N_1 = K.int_shape(G)[0]
        G_gram = get_Gram_matrix(G)
        A_gram = get_Gram_matrix(A)
        sLoss += w*0.25*K.sum(K.square(G_gram - A_gram))/ (N_1 ** 2 * M_1 ** 2)
    return sLoss


In [17]:
def get_total_loss(gImPlaceHolder, aplha=1.0, beta = 10000.0):
    F = get_feature_reps(gImPlaceHolder, layer_names=[cLayerName], model=gModel)[0]
    Gs = get_feature_reps(gImPlaceHolder, layer_names=sLayerNames, Model=gModel)
    contentLoss = get_content_loss(F, P)
    styleLoss = get_style_loss(ws, Gs, As)
    totalLoss = alpha*contentLoss + beta*styleLoss
    return totalLoss

In [18]:
def calculate_loss(gImArr):
    if gImArr.shape != (1, targetWidth, targetWidth, 3):
        gImArr = gImArr.reshape((1, targetWidth, targetHeight, 3))
    loss_fcn =K.function([gModel.input], [get_total_loss(gModel.input)])
    return loss_fcn([gImArr])[0].astype('float64')


In [20]:
def get_grad(gImArr):
        if gIamArr.shape != (1, targetWidth, targetHeight, 3):
            gImArr = gImArr.reshape((1, targetWidth, targetHeight, 3))
        grad_fcn = K.function([gModel.input], K.gradients(get_total_loss(gModel.input), [gModel.input]))
        grad = grad_fcn((gImArr))[0].flatten().astype('float64')
        return grad
    

In [22]:
from keras.applications import VGG16
from scipy.optimize import fmin_l_bfgs_b

In [24]:
tf_session = K.get_session()


In [25]:
cModel = VGG16(include_top=False, weights='imagenet', input_tensor=cImArr)


In [26]:
sModel = VGG16(include_top=False, weights='imagenet', input_tensor=sImArr)


In [27]:
gModel = VGG16(include_top=False, weights='imagenet', input_tensor=gImPlaceholder)


In [32]:
cLayerName = 'block4_conv2'
sLayerNames = [
                'block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                ]

In [33]:
P = get_feature_reps(x=cImArr, layer_names=[cLayerName], model=cModel)[0]

In [34]:
As = get_feature_reps(x=sImArr, layer_names=sLayerNames, model = sModel)

In [35]:
ws=np.ones(len(sLayerNames))/float(len(sLayerNames))

In [36]:
iterations = 600
x_val = gIm0.flatten()

In [38]:
xopt, f_val, info = fmin_l_bfgs_b(calculate_loss, x_val, fprime=get_grad, maxiter=iterations, disp=True)

TypeError: get_feature_reps() got an unexpected keyword argument 'Model'